<a href="https://colab.research.google.com/github/rehan-kapadia/Trading/blob/main/Trading_Library_V3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Trading System Module

This module implements a production‐quality backtesting engine for a single ticker
against a benchmark index based on the following trading rules:

Entry Rules:
1. Primary Trendline Breakout Entry:
   - Within a lookback of 250 bars, identify swing(2) highs.
   - If three or more swing(2) highs exist, compute a candidate trendline (using linear regression).
   - An entry is signaled when the current close exceeds the candidate trendline by a buffer of 0.1%.

2. Two‐Swing2 Entry with EMA Conditions:
   - If only two swing(2) highs exist in the window and they occurred when the close > 50 EMA,
     and thereafter the price never closed below 50 EMA, with the latest bar satisfying:
         EMA_20 > EMA_50 > EMA_200,
     then an entry is signaled at breakout above the second swing(2) high (plus buffer).

3. Swing(5) Entry:
   - If the most recent swing(5) high is identified (using 5 bars on each side) and at least 8 bars
     have passed with the close never dropping below the 20 EMA, then an entry is signaled when price
     breaks above the swing(5) high by 0.1%.

Exit Rules:
1. Initial Stop Loss is set at the closest swing(1) low to the entry price with a 0.2% buffer.
2. Trailing stops update when profit equals the risk, and only if a bar closes above the previous high
   preceding the swing(1) low.
3. A safety exit occurs if price falls below a formed swing(5) low, or if the price closes below the 20 EMA
   twice consecutively.
4. Profit taking: optionally exit (or partially exit) when the profit target (e.g., 3× risk) is reached.
5. Position sizing is determined based on risking 1% of capital with a maximum allocation of 20%.

For each trade, the backtest identifies the entry date, entry price, calculated stop loss, and exit date/price.

The module logs all trades and prints the most recent three trade entries/exits, then plots a chart
for each of the three most recent trades with candlestick data, swing points, and drawn trendlines.
"""

import yfinance as yf
import pandas as pd
import numpy as np
import logging
import time
import sys
from typing import List, Dict, Optional, Tuple
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import LinearRegression
from itertools import combinations

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - [%(levelname)s] - %(message)s',
                    handlers=[
                        logging.StreamHandler(sys.stdout),
                        logging.FileHandler("trading_system.log")
                    ])
logger = logging.getLogger(__name__)

# -----------------------------------------------------------------------------
# 1. Data Download
# -----------------------------------------------------------------------------
def download_data_pair(ticker: str, index: str, start_date: str, end_date: str, interval: str = "1d") -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Download historical data for a ticker and an index using yfinance.
    Returns two DataFrames with a flat column structure.
    """
    def download_single(symbol: str) -> pd.DataFrame:
        df = yf.download(symbol, start=start_date, end=end_date, interval=interval, multi_level_index=False)
        if not df.empty:
            df.reset_index(inplace=True)
        return df

    stock_df = download_single(ticker)
    index_df = download_single(index)
    return stock_df, index_df

# -----------------------------------------------------------------------------
# 2. Indicator Calculations
# -----------------------------------------------------------------------------
def compute_ema(series: pd.Series, period: int) -> pd.Series:
    if period < 1 or series.empty:
        raise ValueError("Invalid input for EMA computation")
    return series.ewm(span=period, adjust=False).mean()

def compute_swing_values(series: pd.Series, max_n: int = 5, swing_type: str = 'low') -> pd.Series:
    if swing_type not in ['low', 'high'] or series.empty or len(series) <= 2*max_n:
        raise ValueError("Invalid input for swing calculation")
    arr = series.to_numpy()
    swing_vals = np.zeros(len(arr), dtype=int)
    for i in range(max_n, len(arr) - max_n):
        val = 0
        for n in range(1, max_n + 1):
            if swing_type == 'low':
                if all(arr[i] < arr[i - k] for k in range(1, n + 1)) and all(arr[i] < arr[i + k] for k in range(1, n + 1)):
                    val = n
                else:
                    break
            else:
                if all(arr[i] > arr[i - k] for k in range(1, n + 1)) and all(arr[i] > arr[i + k] for k in range(1, n + 1)):
                    val = n
                else:
                    break
        swing_vals[i] = val
    return pd.Series(swing_vals, index=series.index)

def compute_relative_strength(stock_df: pd.DataFrame, index_df: pd.DataFrame, window: int = 20) -> pd.Series:
    if 'Low' not in stock_df.columns or 'Low' not in index_df.columns:
        raise ValueError("Missing 'Low' column")
    stock_df = stock_df.copy().sort_index()
    index_df = index_df.copy().sort_index().reindex(stock_df.index, method='ffill')
    stock_prior = stock_df['Low'].shift(1).rolling(window=window, min_periods=window).min()
    index_prior = index_df['Low'].shift(1).rolling(window=window, min_periods=window).min()
    broke_stock = stock_df['Low'] < stock_prior
    broke_index = index_df['Low'] < index_prior
    return broke_index & (~broke_stock)

def compute_rsi(series: pd.Series, period: int = 14) -> pd.Series:
    if series.empty or period < 1:
        raise ValueError("Invalid input for RSI")
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(0)

def compute_all_indicators(stock_df: pd.DataFrame, index_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Compute EMA, RSI, and swing values for stock and index data.
    Adds columns:
      - EMA_20, EMA_50, EMA_200, RSI_14,
      - Swing_Low_Value, Swing_High_Value.
    Also computes relative strength flag in the stock_df.
    """
    for period in [20, 50, 200]:
        stock_df[f"EMA_{period}"] = compute_ema(stock_df["Close"], period)
    stock_df["RSI_14"] = compute_rsi(stock_df["Close"], 14)
    stock_df["Swing_Low_Value"] = compute_swing_values(stock_df["Low"], max_n=5, swing_type='low')
    stock_df["Swing_High_Value"] = compute_swing_values(stock_df["High"], max_n=5, swing_type='high')
    index_df["Swing_Low_Value"] = compute_swing_values(index_df["Low"], max_n=5, swing_type='low')
    index_df["Swing_High_Value"] = compute_swing_values(index_df["High"], max_n=5, swing_type='high')
    stock_df["IsStronger"] = compute_relative_strength(stock_df, index_df, window=20)
    return stock_df, index_df

# -----------------------------------------------------------------------------
# 3. Entry/Exit Signal Functions
# -----------------------------------------------------------------------------
def calculate_trendline(df: pd.DataFrame, lookback: int = 250, tolerance: float = 0.01) -> Optional[Dict[str, any]]:
    """
    Calculate candidate trendline from swing(2) highs over the lookback period using LinearRegression.
    Returns a dict with 'slope', 'intercept', 'r_squared', 'start_date', 'end_date',
    'start_price', 'end_price' if at least 3 swing(2) points exist, otherwise None.
    """
    recent = df.tail(lookback)
    swing_points = recent[recent["Swing_High_Value"] == 2]
    if len(swing_points) < 3:
        return None
    x = swing_points['Date'].map(pd.Timestamp.toordinal).values.reshape(-1, 1)  # Reshape for sklearn
    y = swing_points['High'].values

    # Create and fit the LinearRegression model
    model = LinearRegression()
    model.fit(x, y)

    # Get slope, intercept, and R-squared
    slope = model.coef_[0]
    intercept = model.intercept_
    r_squared = model.score(x, y)

    start_date = swing_points['Date'].iloc[0]
    end_date = swing_points['Date'].iloc[-1]
    start_price = slope * start_date.toordinal() + intercept
    end_price = slope * end_date.toordinal() + intercept

    return {
        'slope': slope,
        'intercept': intercept,
        'r_squared': r_squared,  # Include R-squared in the output
        'start_date': start_date,
        'end_date': end_date,
        'start_price': start_price,
        'end_price': end_price
    }


def validate_swing2_entry(df: pd.DataFrame, lookback: int = 250, buffer: float = 0.001):
    """
    Identifies valid Swing2 entries with EMA conditions within a lookback period.

    Args:
        df (pd.DataFrame): DataFrame containing columns ['Date', 'High', 'Close', 'Swing_High_Value', 'EMA_20', 'EMA_50', 'EMA_200'].
        lookback (int): Lookback window in bars for finding swing(2) highs.
        buffer (float): Percentage buffer added to the swing high to determine the entry price.

    Returns:
        dict or None: Details of the valid trendline and entry information if conditions are met, else None.
    """
    recent_df = df.tail(lookback).reset_index(drop=True)
    swing_highs = recent_df[(recent_df['Swing_High_Value'] == 2) & (recent_df['Close'] > recent_df['EMA_50'])]

    if len(swing_highs) < 3:
        return None  # Insufficient valid swing points

    # Generate trendlines from combinations of swing points
    for i in range(len(swing_highs) - 1):
        for j in range(i + 1, len(swing_highs)):
            point_a = swing_highs.iloc[i]
            point_b = swing_highs.iloc[j]

            date_a_ord = pd.Timestamp(point_a['Date']).toordinal()
            date_b_ord = pd.Timestamp(point_b['Date']).toordinal()

            # Compute linear trendline parameters (y = mx + c)
            slope = (point_b['High'] - point_a['High']) / (date_b_ord - date_a_ord)
            intercept = point_a['High'] - slope * date_a_ord

            points_touched = 2
            breach = False

            # Check intermediate swing highs
            intermediate_points = swing_highs.iloc[i + 1:j]
            for _, mid_point in intermediate_points.iterrows():
                mid_ord = pd.Timestamp(mid_point['Date']).toordinal()
                expected_trendline_high = slope * mid_ord + intercept

                if np.isclose(mid_point['High'], expected_trendline_high, atol=0.01):
                    points_touched += 1
                elif mid_point['Close'] > expected_trendline_high:
                    breach = True
                    break

            if breach or points_touched < 3:
                continue  # Invalid trendline

            # Verify no close breaches trendline between points A and B
            between_points_df = recent_df[(recent_df['Date'] >= point_a['Date']) & (recent_df['Date'] <= point_b['Date'])]
            for _, bar in between_points_df.iterrows():
                bar_ord = pd.Timestamp(bar['Date']).toordinal()
                trendline_value = slope * bar_ord + intercept
                if bar['Close'] > trendline_value:
                    breach = True
                    break

            if breach:
                continue

            # Latest bar conditions
            latest_bar = recent_df.iloc[-1]
            if not (latest_bar['EMA_20'] > latest_bar['EMA_50'] > latest_bar['EMA_200']):
                continue  # EMA stacking condition not met

            entry_price = point_b['High'] * (1 + buffer)
            if latest_bar['High'] >= entry_price:
                return {
                    "start_date": point_a['Date'],
                    "end_date": point_b['Date'],
                    "points_touched": points_touched,
                    "entry_price": entry_price
                }

    return None



def validate_swing5_entry(df: pd.DataFrame, min_bars: int = 8, debug: int = 0) -> Optional[tuple]:
    """
    Validate a swing(5) entry using the following conditions:

    1) Identify the most recent swing(5) high.
    2) From the bar immediately after this swing(5) high until a breakout occurs:
         - All bars must have:
             a) Their High below the swing(5) high price.
             b) Their Close above the corresponding 20 EMA.
         - Additionally, at least min_bars bars must pass since the swing(5) high.
    3) A breakout is defined as the first bar (after the min_bars period) where the Close exceeds
       the swing(5) high price by at least 0.01% (i.e. candidate_price * 1.0001).
    4) The entry is executed on the day of the breakout at the breakout price.

    Returns:
        A tuple (entry_date, entry_price) if all conditions are met, otherwise None.
    """
    # Step 1: Identify the most recent swing(5) high.
    swing5 = df[df["Swing_High_Value"] >= 5]
    if swing5.empty:
        if debug:
            print("No swing(5) high found.")
        return None

    # Filter swing5 to include only those at least 3 bars old
    swing5_filtered = swing5[swing5.index <= len(df) - 3]

    if swing5_filtered.empty:
        if debug:
            print("No swing(5) high found that is at least 8 bars old.")
        return None
    candidate = swing5_filtered.iloc[-1]  # Most recent swing(5) high row that is at least 3 bars old.
    candidate_price = candidate["High"]  # Reference price from the swing(5) high.
    candidate_idx = df.index.get_loc(swing5_filtered.index[-1])
    if debug:
        print(f"Candidate swing(5) high found at index {candidate_idx} with price {candidate_price} on {candidate['Date']}.")

    # *** New Check for Distance from Current Date ***
    if len(df) - candidate_idx < min_bars:
        if debug:
            print("Latest swing(5) high is too close to the current date.")
        return None

    if not (df["EMA_20"].iloc[-1] > df["EMA_50"].iloc[-1] > df["EMA_200"].iloc[-1]):
        if debug:
            print("EMA conditions not met.")
        return None

    breakout_idx = None

    # Step 2 & 3: Iterate over bars after the candidate swing(5) high.
    for i in range(candidate_idx + 1, len(df)):
        row = df.iloc[i]
        if debug:
            print(f"Row {i}: Date {row['Date']}, High {row['High']}, Close {row['Close']}, EMA_20 {row['EMA_20']} -- Candidate Price {candidate_price}")

        if not (row["EMA_20"] > row["EMA_50"] > row["EMA_200"]):
            if debug:
                print("EMA conditions not met.")
            return None

        # Check for breakout condition: the Close exceeds swing high by at least 0.01%.
        if row["High"] > candidate_price * 1.0001:
            if debug:
                print(f"Breakout detected at row {i}: High {row['High']} exceeds breakout threshold {candidate_price * 1.0001}.")
            # Ensure that at least min_bars have passed since the swing(5) high.
            if i - candidate_idx < min_bars:
                if debug:
                    print(f"Breakout occurred too early. Only {i - candidate_idx} bars passed; need at least {min_bars}.")
                return None
            breakout_idx = i
            break

        # For bars before breakout, enforce that:
        # - The High remains below the swing(5) high price.
        # - The Close remains above the 20 EMA.
        if not (row["High"] < candidate_price and row["Close"] >= row["EMA_20"]):
            if debug:
                print(f"Condition failed at row {i}: High {row['High']} is not below Candidate Price {candidate_price} "
                      f"or Close {row['Close']} is below EMA_20 {row['EMA_20']}.")
            return None

    if breakout_idx is None:
        if debug:
            print("No breakout found after candidate swing(5) high.")
        return None

    # Step 4: Execute the entry on the breakout day at the breakout price.
    entry_date = df.iloc[breakout_idx]["Date"]
    entry_price = df.iloc[breakout_idx]["Close"]
    if debug:
        print(f"Entry executed on {entry_date} at price {entry_price}.")
    return entry_date, entry_price



def calculate_stop_loss(df: pd.DataFrame, entry_index: int, entry_price: int, buffer: float = 0.002) -> float:
    """
    Calculate the initial stop loss for an entry based on the closest swing(1) low.
    For simplicity, we use the minimum low in a lookback window ending at the entry.
    """
    lookback = 20
    start_idx = max(0, entry_index - lookback)
    window = df.iloc[start_idx:entry_index + 1]
    swing1 = window[window["Swing_Low_Value"] == 1]

    if not swing1.empty:
        breakout_price = df.iloc[entry_index]["Close"]

        # Calculate price differences and filter for negative differences (swing lows below breakout price)
        price_diffs = swing1["Low"] - breakout_price
        negative_diffs = price_diffs[price_diffs < 0]

        if not negative_diffs.empty:
            # Find the index of the swing(1) low with the smallest negative price difference
            nearest_swing_idx = negative_diffs.idxmin()
            candidate = swing1.loc[nearest_swing_idx, "Low"]
        else:
            # If no swing(1) lows are below the breakout price, use the minimum low within the window
            candidate = window["Low"].min()
    else:
        candidate = window["Low"].min()

    return candidate * (1 - buffer)

def determine_exit(stock_df: pd.DataFrame, entry_idx: int, entry_price: float, initial_stop: float,
                   buffer: float = 0.0, debug: int = 0) -> Tuple[pd.Timestamp, float, str]:
    """
    Determine the exit for a trade starting from entry_idx using trailing stop logic.

    Existing logic:
      - Breakeven: If a bar's high reaches entry_price + risk (1× risk), move stop to breakeven (entry_price).
      - Swing(5) low update: Update stop if a bar qualifies as a swing(5) low (Swing_Low_Value >= 5) and its low
        is above the initial stop (pre-breakeven) or above entry (post-breakeven).
      - Swing(1) trailing stop: Capture a pending temporary high until a swing(1) candidate is detected (by Swing_Low_Value==1
        or if the current low is lower than the previous bar’s low). Then, once a later bar closes above the locked temporary
        high, update stop to the candidate low.
      - EMA20 breakdown and profit target conditions.

    New conditions to be integrated:
      1) Gap Condition:
         - If (previous bar's high - current bar's low) / previous bar's high >= 1%,
           update stop loss to (previous bar's close - buffer).
      2) Inside Bar Condition:
         - If the current bar is an inside bar (i.e. its high is lower than the previous bar's high and its low is higher
           than the previous bar's low), record the previous bar as a candidate.
         - In subsequent bars, if the close exceeds the candidate’s high, update the stop loss to (candidate’s low - buffer).

    Returns:
       A tuple (exit_date, exit_price, exit_reason).
    """
    risk = entry_price - initial_stop
    current_stop = initial_stop
    breakeven_set = False

    # For swing(1) trailing stop logic:
    pending_swing1_candidate = None  # Candidate low from a swing(1) formation.
    pending_temp_high = None         # Temporary high to lock in until a candidate is detected.

    # For inside bar condition:
    inside_bar_candidate = None  # Stores the previous (outside) bar info when an inside bar is detected.

    target = entry_price + 3 * risk  # Example profit target.
    consecutive_EMA20_breaks = 0

    if debug:
        print("Starting determine_exit:")
        print(f"  Entry Price: {entry_price}, Initial Stop: {initial_stop}, Risk: {risk}, Target: {target}")

    for i in range(entry_idx + 1, len(stock_df)):
        row = stock_df.iloc[i]
        date = row["Date"]
        high = row["High"]
        low = row["Low"]
        close = row["Close"]
        open = row["Open"]

        if debug:
            print(f"\nProcessing row {i} - Date: {date}")
            print(f"  High: {high}, Low: {low}, Close: {close}, Current Stop: {current_stop}")

        # Only if a previous bar exists (i > entry_idx+1):
        if i > entry_idx + 1:
            prev = stock_df.iloc[i-1]

            # (A) Gap Condition:
            gap_pct = (prev["High"] - low) / prev["High"]
            if gap_pct >= 0.01:
                new_stop = prev["Close"] - buffer
                if new_stop > current_stop:
                    current_stop = new_stop
                    if debug:
                        print(f"  Gap condition triggered: Prev High = {prev['High']}, Current Low = {low} => Gap = {gap_pct*100:.2f}%")
                        print(f"  Updating current_stop to Prev Close - buffer: {prev['Close']} - {buffer} = {current_stop}")

            # (B) Inside Bar Condition:
            # Check if current bar is completely inside the previous bar.
            if high < prev["High"] and low > prev["Low"]:
                inside_bar_candidate = prev
                if debug:
                    print(f"  Inside bar detected. Candidate outside bar: High = {prev['High']}, Low = {prev['Low']}")
            # If an inside bar candidate exists, check if current close exceeds its high.
            if inside_bar_candidate is not None:
                if close > inside_bar_candidate["High"]:
                    new_stop = inside_bar_candidate["Low"] - buffer
                    if new_stop > current_stop:
                        current_stop = new_stop
                        if debug:
                            print(f"  Inside bar condition validated: Close {close} > Candidate Outside Bar High {inside_bar_candidate['High']}")
                            print(f"  Updating current_stop to Candidate Outside Bar Low - buffer: {inside_bar_candidate['Low']} - {buffer} = {current_stop}")
                    # Once used, reset the candidate.
                    inside_bar_candidate = None

        # Existing condition: Breakeven adjustment.
        if not breakeven_set and high >= entry_price + risk:
            current_stop = max(current_stop, entry_price)
            breakeven_set = True
            if debug:
                print(f"  Breakeven triggered: High {high} reached >= {entry_price + risk}. Setting current_stop to {entry_price}")

        # Existing condition: Update pending temporary high for swing(1) candidate.
        if pending_swing1_candidate is None:
            if pending_temp_high is None or high > pending_temp_high:
                pending_temp_high = high
                if debug:
                    print(f"  Updated pending_temp_high to {pending_temp_high}")

        # Existing condition: Detect swing(1) candidate.
        if pending_swing1_candidate is None:
            if (row["Swing_Low_Value"] == 1) or (i > 0 and low < stock_df.iloc[i-1]["Low"]):
                pending_swing1_candidate = low
                if debug:
                    print(f"  Detected swing(1) candidate with low = {pending_swing1_candidate}")
                    print(f"  Locked pending_temp_high = {pending_temp_high}")

        # Existing condition: Validate the pending swing(1) candidate.
        if pending_swing1_candidate is not None and close > pending_temp_high:
            if pending_swing1_candidate > current_stop:
                if debug:
                    print(f"  Swing(1) candidate validated: Close {close} > locked temp high {pending_temp_high}.")
                    print(f"  Updating current_stop from {current_stop} to {pending_swing1_candidate}")
                current_stop = pending_swing1_candidate
            pending_swing1_candidate = None
            pending_temp_high = None

        # Existing condition: Swing(5) low candidate update.
        if row["Swing_Low_Value"] >= 5:
            candidate_stop = low
            if debug:
                print(f"  Swing(5) low candidate detected with low = {candidate_stop}")
            if breakeven_set:
                if candidate_stop > entry_price and candidate_stop > current_stop:
                    current_stop = candidate_stop
                    if debug:
                        print(f"  Updating current_stop to {candidate_stop} (post breakeven candidate)")
            else:
                if candidate_stop > initial_stop and candidate_stop > current_stop:
                    current_stop = candidate_stop
                    if debug:
                        print(f"  Updating current_stop to {candidate_stop} (pre breakeven candidate)")

        # Exit Conditions:
        # (a) If current close breaches the stop loss, exit.
        if close < current_stop:
            if debug:
                print(f"  Exit triggered: Close {close} is below current_stop {current_stop}.")
            if open < current_stop:
                return (date, open, "Trailing Stop Loss Hit")
            else:
                return (date, current_stop, "Trailing Stop Loss Hit")

        # (b) EMA20 breakdown: if close < EMA_20 for two consecutive bars.
        if close < row["EMA_20"]:
            consecutive_EMA20_breaks += 1
            if debug:
                print(f"  EMA20 breakdown count increased to {consecutive_EMA20_breaks}.")
        else:
            if consecutive_EMA20_breaks > 0 and debug:
                print("  EMA20 breakdown count reset to 0.")
            consecutive_EMA20_breaks = 0
        if consecutive_EMA20_breaks >= 2:
            if debug:
                print(f"  Exit triggered: {consecutive_EMA20_breaks} consecutive EMA20 breakdowns.")
            return (date, close, "EMA20 Breakdown")

        # (c) Profit target condition.
        if close >= target:
            if debug:
                print(f"  Profit target reached: Close {close} >= target {target}.")
            return (date, target, "Profit Target Hit")

    # If no exit condition is met, exit at the last available bar.
    last = stock_df.iloc[-1]
    if debug:
        print(f"Exiting at End of Data: Date {last['Date']}, Close {last['Close']}.")
    return (last["Date"], last["Close"], "End of Data")




# -----------------------------------------------------------------------------
# 4. Backtesting Engine: Entry and Exit Pair Identification
# -----------------------------------------------------------------------------


# Assuming you already have:
#   - validate_swing2_entry (the new function created for Swing(2) entries)
#   - validate_swing5_entry (your existing function for Swing(5) entries)
#   - download_data_pair, compute_all_indicators, calculate_stop_loss, determine_exit, etc.

def backtest_trading(
    ticker: str,
    index: str,
    start_date: str,
    end_date: str,
    interval: str = "1d",
    debug: int = 0
) -> pd.DataFrame:
    """
    Modified Backtesting function that checks for both Swing(2) and Swing(5) entries.

    1) Downloads data and computes indicators for the stock and index.
    2) Iterates through each bar:
       - If not in a trade, checks:
         a) validate_swing2_entry
         b) validate_swing5_entry
       - If in a trade, determines exit using your exit logic.
    3) Stores all trades and returns them as a DataFrame.

    Parameters:
        ticker (str): Stock symbol
        index (str): Benchmark index symbol
        start_date (str): Start of historical data
        end_date (str): End of historical data
        interval (str): Frequency of bars (default '1d')
        debug (int): Debug level (0=off, 1=verbose)

    Returns:
        pd.DataFrame: DataFrame of trades with columns:
            [Entry_Date, Entry_Price, Stop_Loss, Exit_Date, Exit_Price, Profit, Entry_Type, Exit_Reason].
    """

    # 1) DOWNLOAD AND PREPARE DATA
    stock_df, index_df = download_data_pair(ticker, index, start_date, end_date, interval)
    if stock_df.empty or index_df.empty:
        raise ValueError("One or both dataframes are empty")

    # Ensure 'Date' in datetime
    stock_df["Date"] = pd.to_datetime(stock_df["Date"])
    index_df["Date"] = pd.to_datetime(index_df["Date"])

    # Compute technical indicators
    stock_df, index_df = compute_all_indicators(stock_df, index_df)

    trades = []
    in_trade = False
    entry_info = {}
    entry_idx = None

    # 2) MAIN BACKTEST LOOP
    # Start from where indicators are valid (e.g. after computing EMAs)
    start_bar = 25  # or whichever minimum offset you need
    for i in range(start_bar, len(stock_df)):
        row = stock_df.iloc[i]

        if not in_trade:
            # 2a) If not in a trade, check for new entry signal
            df_up_to_current = stock_df.iloc[:i+1]  # Data up to current bar

            # Try Swing(2) entry
            swing2_signal = validate_swing2_entry(df_up_to_current, lookback=250, buffer=0.001)
            if swing2_signal is not None:
                # We have a Swing(2) entry
                in_trade = True
                entry_idx = i

                entry_price = swing2_signal["entry_price"]
                stop_loss = calculate_stop_loss(stock_df, i, entry_price, buffer=0.002)

                entry_info = {
                    "Entry_Date": row["Date"].date(),
                    "Entry_Price": entry_price,
                    "Stop_Loss": stop_loss,
                    "Entry_Type": "Swing2 Entry",
                    "Details": swing2_signal  # store additional debug info if desired
                }

                if debug:
                    print(f"Swing(2) Entry triggered on {entry_info['Entry_Date']} at {entry_price:.2f}")

            else:
                # If no Swing(2) signal, check Swing(5)
                swing5_signal = validate_swing5_entry(df_up_to_current, min_bars=8, debug=debug)
                if swing5_signal is not None:
                    in_trade = True
                    entry_idx = i

                    # swing5_signal typically returns (entry_date, entry_price)
                    swing5_date, swing5_price = swing5_signal
                    stop_loss = calculate_stop_loss(stock_df, i, swing5_price, buffer=0.002)

                    entry_info = {
                        "Entry_Date": swing5_date.date(),
                        "Entry_Price": swing5_price,
                        "Stop_Loss": stop_loss,
                        "Entry_Type": "Swing5 Entry",
                        "Details": {}
                    }

                    if debug:
                        print(f"Swing(5) Entry triggered on {entry_info['Entry_Date']} at {swing5_price:.2f}")

        else:
            # 2b) If already in a trade, check for exit conditions
            exit_date, exit_price, exit_reason = determine_exit(
                stock_df, entry_idx, entry_info["Entry_Price"],
                entry_info["Stop_Loss"], debug=debug
            )
            if exit_date is not None:
                # We have an exit
                profit = exit_price - entry_info["Entry_Price"]
                trade = {
                    "Entry_Date": entry_info["Entry_Date"],
                    "Entry_Price": entry_info["Entry_Price"],
                    "Stop_Loss": entry_info["Stop_Loss"],
                    "Exit_Date": exit_date,
                    "Exit_Price": exit_price,
                    "Profit": profit,
                    "Entry_Type": entry_info["Entry_Type"],
                    "Exit_Reason": exit_reason
                }
                trades.append(trade)

                if debug:
                    print(f"Exit on {exit_date} at {exit_price:.2f}, reason: {exit_reason}, profit: {profit:.2f}")

                in_trade = False
                entry_info = {}
                entry_idx = None

    # 3) FINALIZE AND RETURN TRADES
    trades_df = pd.DataFrame(trades)
    if not trades_df.empty:
        trades_df.sort_values("Entry_Date", inplace=True)

    return trades_df





# -----------------------------------------------------------------------------
# 5. Trade Visualization
# -----------------------------------------------------------------------------
def plot_trade(ticker: str, df_stock: pd.DataFrame, trade: Dict[str, any]) -> None:
    """
    Plot a candlestick chart for the given trade period along with:
      - Swing points used for entry (vertical dotted red lines)
      - The candidate trendline from the entry decision
      - Markers for entry and exit
    """
    # Filter data for a window from a few bars before entry to a few bars after exit.
    entry_date = trade["Entry_Date"]
    exit_date = trade["Exit_Date"]

    # Convert entry_date to Timestamp for consistent comparison
    entry_date = pd.Timestamp(entry_date)
    exit_date = pd.Timestamp(exit_date)
    plot_df = df_stock[(df_stock["Date"] >= entry_date - pd.Timedelta(days=20)) & (df_stock["Date"] <= exit_date + pd.Timedelta(days=20))].copy()

    fig = go.Figure(data=[go.Candlestick(
            x=plot_df["Date"],
            open=plot_df["Open"],
            high=plot_df["High"],
            low=plot_df["Low"],
            close=plot_df["Close"],
            name="Price"
        )])
    # Overlay swing points (where swing value != 0)
    swing_mask = (plot_df["Swing_High_Value"] != 0) | (plot_df["Swing_Low_Value"] != 0)
    for d in plot_df.loc[swing_mask, "Date"]:
        fig.add_shape(dict(
            type="line",
            x0=d, x1=d,
            yref="paper", y0=0, y1=1,
            line=dict(color="rgba(255,0,0,0.5)", dash="dot")
        ))
    # Mark entry and exit.
    fig.add_trace(go.Scatter(x=[entry_date], y=[trade["Entry_Price"]],
                             mode="markers", marker=dict(color="green", size=12), name="Entry"))
    fig.add_trace(go.Scatter(x=[exit_date], y=[trade["Exit_Price"]],
                             mode="markers", marker=dict(color="red", size=12), name="Exit"))
    # If available, overlay the candidate trendline (from entry details if it exists).
    if "Trendline" in trade.get("Entry_Type", "") or trade["Entry_Type"] in ["Trendline Breakout", "Two-Swing2 Entry"]:
        # For simplicity, re-calculate the candidate trendline on the full plot window.
        tl = calculate_trendline(plot_df, lookback=len(plot_df))
        if tl is not None:
            fig.add_trace(go.Scatter(
                x=[tl["start_date"], tl["end_date"]],
                y=[tl["start_price"], tl["end_price"]],
                mode="lines",
                line=dict(color="blue", width=2),
                name="Candidate Trendline"
            ))
    fig.update_layout(title=f"{ticker} Trade from {entry_date.date()} to {exit_date.date()}",
                      xaxis_title="Date", yaxis_title="Price")
    fig.show()


def plot_swing_candlesticks(df: pd.DataFrame, window: int = 10) -> None:
    """
    For a given DataFrame containing stock prices and indicators,
    identifies the last 5 dates with Swing Low > 0 and the last 5 dates with Swing High > 0.
    For each identified date, it creates a candlestick plot for the period
    from 'window' days before to 'window' days after the date.

    Each plot is labeled with the swing type and its value for that date.

    Parameters:
      df (pd.DataFrame): DataFrame containing columns 'Date', 'Open', 'High', 'Low', 'Close',
                         'Swing_Low_Value', and 'Swing_High_Value'.
      window (int): Number of days before and after the identified date to include in the plot.
    """
    # Ensure the Date column is a datetime object.
    df["Date"] = pd.to_datetime(df["Date"])

    # Identify the last 5 swing low candidates, preserving the swing value.
    swing_low_candidates = (
        df[df["Swing_Low_Value"] > 0][["Date", "Swing_Low_Value"]]
        .drop_duplicates(subset="Date", keep="last")
        .tail(5)
    )

    # Identify the last 5 swing high candidates, preserving the swing value.
    swing_high_candidates = (
        df[df["Swing_High_Value"] > 0][["Date", "Swing_High_Value"]]
        .drop_duplicates(subset="Date", keep="last")
        .tail(5)
    )

    # Function to create a candlestick plot for a given date.
    def plot_for_date(target_date, swing_type, swing_value):
        start_date = target_date - pd.Timedelta(days=window)
        end_date = target_date + pd.Timedelta(days=window)
        subset = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]
        if subset.empty:
            print(f"No data available for plotting around {target_date.date()}.")
            return

        title = f"Candlestick Chart around {swing_type} on {target_date.date()} (Value: {swing_value})"
        fig = go.Figure(data=[go.Candlestick(
            x=subset["Date"],
            open=subset["Open"],
            high=subset["High"],
            low=subset["Low"],
            close=subset["Close"],
            name="Price"
        )])
        fig.update_layout(
            title=title,
            xaxis_title="Date",
            yaxis_title="Price",
            xaxis_rangeslider_visible=False
        )
        fig.show()

    # Create a plot for each swing low candidate.
    for _, row in swing_low_candidates.iterrows():
        plot_for_date(row["Date"], "Swing Low", row["Swing_Low_Value"])

    # Create a plot for each swing high candidate.
    for _, row in swing_high_candidates.iterrows():
        plot_for_date(row["Date"], "Swing High", row["Swing_High_Value"])

# -----------------------------------------------------------------------------
# 6. Main Function to Run the Backtest and Visualization
# -----------------------------------------------------------------------------
def main():
    # Example input: pair of ticker and index.
    ticker = "MSFT"  #MODIFY THIS TO CHANGE TICKER ANALYZED
    index = "SPY"
    start_date = "2000-01-01"
    end_date = "2025-02-06"
    interval = "1d"
    debug = 0

    # Download data.
    stock_df, index_df = download_data_pair(ticker, index, start_date, end_date, interval)
    if stock_df.empty or index_df.empty:
        logger.error("Data download failed.")
        return

    # Compute technical indicators.
    stock_df, index_df = compute_all_indicators(stock_df, index_df)
    # Ensure that the Date column is in datetime format.
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
    index_df['Date'] = pd.to_datetime(index_df['Date'])

    #swing5_high_rows = stock_df[stock_df["Swing_High_Value"] == 5]
    #swing5_high_rows.to_csv(f'{ticker}_Swing(5)_High.csv', index=False)
    #print(swing5_high_rows.tail(20))

    #plot_swing_candlesticks(stock_df)

    # Run backtest.
    trades_df = backtest_trading(ticker, index, start_date, end_date, interval, debug)
    if trades_df.empty:
        logger.info("No trades identified.")
        return
    # Log all trades.
    trades_df.to_csv(f"trades_log_{ticker}.csv", index=False)
    print(f"All trades logged to trades_log_{ticker}.csv")


    # Print the most recent three trade records.
    recent_trades = trades_df.sort_values("Entry_Date").tail(15)
    logger.info("Most recent 15 trades:")
    logger.info(recent_trades.to_string(index=False))

    # Plot charts for the most recent three trades.
    for _, trade in recent_trades.iterrows():
        plot_trade(ticker, stock_df, trade)

if __name__ == "__main__":
    main()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


All trades logged to trades_log_MSFT.csv


In [1]:
pip install itertools

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools
